In [1]:
#Import dependencies 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from sqlalchemy import create_engine 
import pprint  

# Import API key
api_key = "j0I1nLBcvM9JVNmM8Ct2fWexeenT6rzjPtToIgIm" 

In [2]:
url = "https://api.usa.gov/crime/fbi/sapi/"

state_abbrv = "CA"
departments = [] 
department_name = []
county_name = []
#state_abbr = [] 
ori = []
assault_a = []
sex_offense = []
man_neg = []
murder = []
rape = []
robbery = []
assault_s = []
ht_c = []
ht_i = []
total = []

In [3]:
response = json.loads(requests.get(f'{url}api/agencies/byStateAbbr/{state_abbrv}?API_KEY={api_key}').text)

for i in range(len(response["results"])):
    if "orange" in response["results"][i]["county_name"].lower():
        departments.append(response["results"][i])
        department_name.append(response["results"][i]["agency_name"])
        county_name.append(response["results"][i]["county_name"])
        ori.append(response["results"][i]["ori"])
        #state_abbr.append(response["results"][i]["state_abbr"])
        
#print(departments)
print(json.dumps(departments, indent=4, sort_keys=True))

[
    {
        "agency_name": "Orange County Sheriff's Office",
        "agency_type_name": "County",
        "county_name": "ORANGE",
        "division_name": "Pacific",
        "latitude": 33.675687,
        "longitude": -117.777207,
        "nibrs": false,
        "nibrs_start_date": null,
        "ori": "CA0300000",
        "region_desc": "Region IV",
        "region_name": "West",
        "state_abbr": "CA",
        "state_name": "California"
    },
    {
        "agency_name": "Anaheim Police Department",
        "agency_type_name": "City",
        "county_name": "ORANGE",
        "division_name": "Pacific",
        "latitude": 33.675687,
        "longitude": -117.777207,
        "nibrs": false,
        "nibrs_start_date": null,
        "ori": "CA0300100",
        "region_desc": "Region IV",
        "region_name": "West",
        "state_abbr": "CA",
        "state_name": "California"
    },
    {
        "agency_name": "Brea Police Department",
        "agency_type_name": "City"

In [4]:
#Creating Police Dept. Df 
police_df = pd.DataFrame({"Dept.": department_name,
                        "County": county_name,
                        #"State": state_abbr
                        "Ori": ori,
                        })
police_df

,Dept.,County,Ori
0,Orange County Sheriff's Office,ORANGE,CA0300000
1,Anaheim Police Department,ORANGE,CA0300100
2,Brea Police Department,ORANGE,CA0300200
3,Buena Park Police Department,ORANGE,CA0300300
4,Costa Mesa Police Department,ORANGE,CA0300400
5,Cypress Police Department,ORANGE,CA0300500
6,La Palma Police Department,ORANGE,CA0300600
7,Fountain Valley Police Department,ORANGE,CA0300700
8,Fullerton Police Department,ORANGE,CA0300800
9,Garden Grove Police Department,ORANGE,CA0300900


In [23]:
# Remove all non police department listings (i.e. universities, highway patrol, etc.)
cities = []
for i,pd in enumerate(police_df['Dept.']):
    if 'Police Department' in pd:
        cities.append(pd.split(' Police')[0])
    else:
        police_df = police_df.drop(i)
police_df['City'] = cities

In [25]:
#Remove county since they are all the same
del police_df['County']

In [28]:
#Save the police df as a csv
police_df.to_csv('Data/police_df.csv')

In [5]:
crime_data = []


for i in range(len(departments)):
    response = json.loads(requests.get(f'{url}api/arrest/agencies/offense/{departments[i]["ori"]}/violent_crime/2019/2019?API_KEY={api_key}').text)
    crime_data.append(response)

print(json.dumps(crime_data[30], indent=4, sort_keys=True))
for i in range(len(crime_data) - 1):
    if len(crime_data[i]["data"]) == 0:
        assault_a.append("N/A")
        sex_offense.append("N/A")
        man_neg.append("N/A")
        murder.append("N/A")
        rape.append("N/A")
        robbery.append("N/A")
        assault_s.append("N/A")
        ht_c.append("N/A")
        ht_i.append("N/A")
        total.append("N/A")
    else:
        assault_a.append(crime_data[i]["data"][0]["value"])
        sex_offense.append(crime_data[i]["data"][1]["value"])
        man_neg.append(crime_data[i]["data"][2]["value"])
        murder.append(crime_data[i]["data"][3]["value"])
        rape.append(crime_data[i]["data"][4]["value"])
        robbery.append(crime_data[i]["data"][5]["value"])
        assault_s.append(crime_data[i]["data"][6]["value"])
        ht_c.append(crime_data[i]["data"][7]["value"])
        ht_i.append(crime_data[i]["data"][8]["value"])
        total.append(assault_a[i] + sex_offense[i] + man_neg[i] + murder[i] + rape[i] + robbery[i] + assault_s[i] + ht_c[i] + ht_i[i])

print(json.dumps(crime_data, indent=4, sort_keys=True)) 

{
    "category": "Arrestee Offense",
    "data": [],
    "keys": [
        "Aggravated Assault",
        "Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)",
        "Manslaughter by Negligence",
        "Murder and Nonnegligent Manslaughter",
        "Rape",
        "Robbery",
        "Simple Assault",
        "Human Trafficking - Commercial Sex Acts",
        "Human Trafficking - Involuntary Servitude"
    ],
    "noun": "Offense",
    "precise_data": [],
    "short_title": "offense",
    "title": "Arrests for Crime Against Person Offenses",
    "ui_restriction": null,
    "ui_type": "basic_table"
}
[
    {
        "category": "Arrestee Offense",
        "data": [
            {
                "data_year": 2019,
                "key": "Aggravated Assault",
                "month_num": 0,
                "value": 183
            },
            {
                "data_year": 2019,
                "key": "Sex Offenses (Except Rape, and Prostitution and Commercialized

In [9]:
#creating Offences Df 

offense_df = pd.DataFrame({"Agg. Assult": assault_a,
                           "Sex Offences":sex_offense,
                           "Manslaughter Neg.": man_neg, 
                           "Murder & Nonneg. Man.": murder,  
                           "Rape": rape,  
                           "Robbery": robbery, 
                           "Simp. Assult": assault_s,  
                           "Humman Traff. (Commercial SA)": ht_c, 
                           "Humman Traff. (Invol. Ser.)": ht_i,
                           "Offences Total": total
                        })
offense_df 

,Agg. Assult,Sex Offences,Manslaughter Neg.,Murder & Nonneg. Man.,Rape,Robbery,Simp. Assult,Humman Traff. (Commercial SA),Humman Traff. (Invol. Ser.),Offences Total
0,183,16,1,3,16,22,160,0,0,401
1,737,113,1,15,23,155,775,0,0,1819
2,24,7,0,0,0,5,86,0,0,122
3,97,21,0,1,4,28,140,0,0,291
4,204,34,0,0,6,45,146,0,0,435
5,48,6,0,1,2,4,29,0,0,90
6,20,3,0,0,0,3,27,0,0,53
7,50,2,0,0,1,10,80,0,0,143
8,308,53,2,2,12,43,541,0,0,961
9,282,43,0,9,18,68,238,0,0,658


In [21]:
#joining Police and Offense df together 

police_df.append(offense_df)

#police_df.merge(offense_df, how='outer')
#left_on=None, right_on=None, left_index=False, right_index=False

,Dept.,County,Ori,Agg. Assult,Sex Offences,Manslaughter Neg.,Murder & Nonneg. Man.,Rape,Robbery,Simp. Assult,Humman Traff. (Commercial SA),Humman Traff. (Invol. Ser.),Offences Total
0,Orange County Sheriff's Office,ORANGE,CA0300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Anaheim Police Department,ORANGE,CA0300100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brea Police Department,ORANGE,CA0300200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Buena Park Police Department,ORANGE,CA0300300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Costa Mesa Police Department,ORANGE,CA0300400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,NaN,NaN,NaN,3,0,0,0,0,1,3,0,0,7
37,NaN,NaN,NaN,36,2,0,0,2,3,28,0,0,71
38,NaN,NaN,NaN,44,0,1,0,1,5,34,0,0,85
39,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0


In [ ]:
police_df.to